In [1]:
import json
import pytorch_lightning as pl



from modelAB import *
from modelCD import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PAD_TOKEN = '<pad>'
EN_TRAIN_PATH = "./../../data/EN/train.json"
EN_DEV_PATH = "./../../data/EN/dev.json"
BERT_PATH = "./../../model/bert-base-cased"
#BERT_PATH = "./model/bert-base-cased"
VERBATLAS_PATH = "./VerbAtlas/VerbAtlas"


cpu
11.3
cuda
11.3


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/michele/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/michele/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def read_dataset(path: str):
    with open(path) as f:
        dataset = json.load(f)

    sentences, labels = {}, {}
    for sentence_id, sentence in dataset.items():
        sentence_id = sentence_id
        sentences[sentence_id] = {
            "words": sentence["words"],
            "lemmas": sentence["lemmas"],
            "pos_tags": sentence["pos_tags"],
            "dependency_heads": [int(head) for head in sentence["dependency_heads"]],
            "dependency_relations": sentence["dependency_relations"],
            "predicates": sentence["predicates"],
        }

        labels[sentence_id] = {
            "predicates": sentence["predicates"],
            "roles": {int(p): r for p, r in sentence["roles"].items()}
            if "roles" in sentence
            else dict(),
        }

    return sentences, labels

In [3]:
'''
early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_f1',  # the value that will be evaluated to activate the early stopping of the model.
    patience=5,
    # the number of consecutive attempts that the model has to raise (or lower depending on the metric used) to raise the "monitor" value.
    verbose=True,  # whether to log or not information in the console.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
)

check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_f1',  # the value that we want to use for model selection.
    verbose=True,  # whether to log or not information in the console.
    save_top_k=1,  # the number of checkpoints we want to store.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
    filename='modelAB_{epoch}-{val_f1:.4f}'
    # the prefix on the checkpoint values. Metrics store by the trainer can be used to dynamically change the name.
)

ab_dm = PredicatesDataModule(
    verbAtlas_path=VERBATLAS_PATH,
    data_train_path=EN_TRAIN_PATH,
    data_dev_path=EN_DEV_PATH,
    data_test_path=EN_DEV_PATH,
    batch_size=8
)

ab_classifier = AB_Model(language="en", hidden2=200, lstm_layers=1, bidirectional=False, p=0.5).to(device)

# the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=20,  # maximum number of epochs.
    gpus=1,  # the number of gpus we have at our disposal.
    callbacks=[early_stopping, check_point_callback]  # the callback we want our trainer to use.
)

# and finally we can let the "trainer" fit the amazon reviews classifier.
trainer.fit(model=ab_classifier, datamodule=ab_dm)

model_path = "../../model/modelAB.ckpt"
#classifier = AB_Model.load_from_checkpoint(model_path, language="en").to(device)




sentences, labels = read_dataset(EN_DEV_PATH)

for idx, key in enumerate(sentences):
    prediction = ab_classifier.predict(sentences[key])
    lab = labels[key]["predicates"]
    print("PREDICTED", len(prediction), prediction)
    print("GROUND TRUTH", len(lab), lab)
#'''

'\nearly_stopping = pl.callbacks.EarlyStopping(\n    monitor=\'val_f1\',  # the value that will be evaluated to activate the early stopping of the model.\n    patience=5,\n    # the number of consecutive attempts that the model has to raise (or lower depending on the metric used) to raise the "monitor" value.\n    verbose=True,  # whether to log or not information in the console.\n    mode=\'max\',  # wheter we want to maximize (max) or minimize the "monitor" value.\n)\n\ncheck_point_callback = pl.callbacks.ModelCheckpoint(\n    monitor=\'val_f1\',  # the value that we want to use for model selection.\n    verbose=True,  # whether to log or not information in the console.\n    save_top_k=1,  # the number of checkpoints we want to store.\n    mode=\'max\',  # wheter we want to maximize (max) or minimize the "monitor" value.\n    filename=\'modelAB_{epoch}-{val_f1:.4f}\'\n    # the prefix on the checkpoint values. Metrics store by the trainer can be used to dynamically change the name.\n

In [4]:
#'''
early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_f1',  # the value that will be evaluated to activate the early stopping of the model.
    patience=5,  # the number of consecutive attempts that the model has to raise (or lower depending on the metric used) to raise the "monitor" value.
    verbose=True,  # whether to log or not information in the console.
    mode='max', # wheter we want to maximize (max) or minimize the "monitor" value.
)

check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_f1',  # the value that we want to use for model selection.
    verbose=True,  # whether to log or not information in the console.
    save_top_k=1,  # the number of checkpoints we want to store.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
    filename='model_CD{epoch}-{val_f1:.4f}'  # the prefix on the checkpoint values. Metrics store by the trainer can be used to dynamically change the name.
)


cd_dm = SentencesDataModule(
    data_train_path=EN_TRAIN_PATH,
    data_dev_path=EN_DEV_PATH,
    data_test_path=EN_DEV_PATH,
    batch_size=16
)
cd_classifier = CD_Model(language="en",p=0.5)
# the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=20,  # maximum number of epochs.
    gpus=1,  # the number of gpus we have at our disposal.
    callbacks=[early_stopping, check_point_callback]  # the callback we want our trainer to use.
)
# and finally we can let the "trainer" fit the amazon reviews classifier.
trainer.fit(model=cd_classifier, datamodule=cd_dm)
model_path = "../../model/modelCD.ckpt"
classifier = CD_Model.load_from_checkpoint(model_path,language="en").to(device)



sentences,labels = read_dataset(EN_DEV_PATH)


for idx,key in enumerate(sentences):
    prediction = cd_classifier.predict(sentences[key])["roles"]
    lab = labels[key]["roles"]
    print("PREDICTED",prediction)
    print("GROUND TRUTH",lab)

#'''

Some weights of the model checkpoint at ../../model/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ../../model/bert-base-cased and are newly initialized: ['bert.encoder

Validation sanity check: 0it [00:00, ?it/s]

/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 whic

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Epoch 59, global step 47459: val_f1 was not in top 1


Validating: 0it [00:00, ?it/s]

Monitored metric val_f1 did not improve in the last 5 records. Best score: 0.954. Signaling Trainer to stop.
Epoch 60, global step 48250: val_f1 was not in top 1
Some weights of the model checkpoint at ../../model/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassi

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)